In [71]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import time
import re
import csv
import pandas as pd
import json

## Get the data

This part was used for downloading all the movies and saving them into single html files

In [2]:
url = 'https://raw.githubusercontent.com/CriMenghini/ADM/master/2019/Homework_3/data/movies2.html'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
# download all the html pages
for i in range(1):
    link = str(soup.select('a')[i].text)
    time.sleep(np.random.randint(1,6))
    movie = str(BeautifulSoup(requests.get(link).text, 'html.parser'))
    f = open('C:/Users/Sasha/Desktop/DS/ADM/HW3/movies/movie_{}.html'.format(i), 'w', encoding="utf-8")
    f.write(movie)

some functions needed to cleare the data

In [2]:
# function needed to remove the quotations, in the form [.], from a string 
def remove_quotes(x):
    # using regex, finding all substrings contained between the characters '[' and ']'
    #'quotes' is a list with all those characters, we loop over all of them, and then delete them from 'x'
    quotes = re.findall(r'\[(.+?)\]', x)
    for i in range(len(quotes)):
        x = x.replace('[{}]'.format(quotes[i]), '')
    return (x)

In [3]:
# this function splits 2 words if they are attached and the second one starts with an uppercase. 
# this happens many times when there are more than 1 names in 'music', 'starring'...
def split_names(a):
    for i in range(1,len(a)):
        # when we encounter an upper case letter and the previous is not a tab, we split the word
        if (a[i].isupper() == True) & (a[i-1] != ' '):
            a = a[:i] + ' ' + a[i:]
    return a

In [4]:
# function neede to get the movie title from the wikipedia page title
def get_title(title):
    # titles can come in this format [...] - (1987 American film) - Wikipedia
    # we clean the title from the final word and everything in the brackets.
    bra = re.findall(r'\((.+?)\)', title)
    for i in range(len(bra)):
         title= title.replace(' ({})'.format(bra[i]), '')
    title = title[:-12]
    return title

Extracting info and making tsv files

 Here we get the information about the movie. For the intro we select all paragraphs until the first 'h2' tag. For the plot we select all paragraphs between the first and second 'h2' tag. In this way we should get always the first 2 sections of the wikipedia page. The texts in the section are then saved in the string variables 'intro' and 'plot'. 
Then we get the infobox informations. The infobox table has class 'infobox vevent', so we get that, we analyse all 'keys' of the table: Directed by, Produced by, Starring..., which elements are the respective names. The tsv files are saved as requested and by adding a last element that is the link to the web page, that will be needed for the search engines.

In [8]:
# open the html files with BeautifulSoup
for j in range(10000):
    movie = open('movies2/movie_{}.html'.format(j-10001), 'r', encoding = 'utf-8')
    movie = BeautifulSoup(movie, 'html.parser')
    # wiki page title
    title = str(movie.title.text)
    # link of the page
    link = movie.find('link', {'rel':'canonical'}).get('href')
    # in many cases the information needed are containde in the following div
    movie = movie.find('div', class_='mw-parser-output') 
    
    # few pages are disambiguations, if we get on of those we skip the iteraction of the for loop
    if title.find('(disambiguation)') != -1:
        continue 
        
    # get the title of the movie
    title = get_title(title)
    
    if movie is not None:
        # find_next() function gives the next element, that can be a 'p', a 'div', 'a', etc ( when not 
        # specified). The variable 'x' represents each of those elements, 
        # and we take them all if their name is 'p', and until we meet the element with name 'h2'
        
        # here we get the intro
        x = movie.p #this is the first paragraph
        if x is None:
            intro = 'NA'
        else:
            intro = ''
            while x.name != 'h2':
                if x.name == 'p':
                    #print(x.text)
                    intro = str(intro) + str(x.text)
                x = x.find_next()
                
        # here we get the plot
        x = x.find_next('p')
        if x is None:
            plot = 'NA'
        else:
            plot = ''
            while x.name != 'h2':
                if x.name == 'p':
                    #print(x.text)
                    plot = plot + str(x.text)
                x = x.find_next()
                
        # remove quotes symbols, [.], from the intro
        intro = remove_quotes(intro)               
        #### Now we extract the information from the Infobox ####
        # ge the infobox
        infobox = movie.find('table', class_ = 'infobox vevent') 
       
        # we check info only if the infobox has some information
        # we proceed only if infobox exists
        if infobox is not None:
            if len(infobox.find_all('th')) != 0:
                # we initialize all the variables associated with the respective infos. If the infobox doesn't contain
                # the information, then it just remains 'NA'.
                director = producer = writer = starring = music = release = runtime = country = language = budget = 'NA'
                film_name = infobox.th.text # the first 'th' element is the title'
                i = 0
                # the 'th' tags contain the 'key' (Directed by, Produced by, etc), and the 'td' the respective names.
                for th in infobox.find_all('th'):
                    if i <= len(infobox.find_all('td')) - 1:
                        info = infobox.find_all('td')[i].text
                        if th.text == 'Directed by':
                            director = info
                        elif th.text == 'Produced by':
                            producer = info
                        elif th.text == 'Written by':
                            writer = info
                        elif th.text == 'Starring':
                            starring = info
                        elif th.text == 'Music by':
                            music = info
                        elif th.text == 'Release date':
                            release = info
                        elif th.text == 'Running time':
                            runtime = info
                        elif th.text == 'Country':
                            country = info
                        elif th.text == 'Language':
                            language = info
                        elif th.text == 'Budget':
                            budget = info
                        i += 1
        output_file = title + '\t' + intro + '\t'+plot+'\t'+director+'\t'+producer+'\t'+writer+'\t'+ starring
        output_file = output_file + '\t'+music+'\t'+release+'\t'+runtime+'\t'+country+'\t'+language+'\t'+budget
        output_file = output_file + '\t' + link
        # we cleare the output_file from the '\n' and quotes, in form [.].
        output_file = output_file.replace('\n', '')
        output_file = remove_quotes(output_file)
        output_file = split_names(output_file)
        with open('tsv_files2/movie_{}.tsv'.format(j), "w", encoding = 'utf-8') as outputter:
            outputter.write(output_file)
        if j % 1000 == 0:
            print ('Status: ',j)

Status:  11000
Status:  12000
Status:  13000
Status:  14000
Status:  15000
Status:  16000
Status:  17000
Status:  19000
Status:  20000


## Search Engines

Imports from the nltk library needed to preprocess the words

In [2]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
import collections
from collections import Counter

Defining the processing function, that from a string will put all letters in lowercase, remove stop words and do the lemming. The last 2 steps are done by using the library nltk

In [3]:
# preprocessing a string, deleting stop words, and lemming
# this is needed for making the 
def processing(string):
    # we use the library nltk to remove stop words and do the lemming
    # convert all in lower case
    string = string.lower()
    # delete all but alphanumeric characters
    string = re.sub(r'\W+', ' ', string)
    # ge the set of stopwords
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(string) 
    # return string without stop words and after lemming
    return np.array(list([PorterStemmer().stem(x) for x in word_tokens if not x in stop_words]))   

We will use 2 dictionaries for the vocabulary and then inverted index

In [ ]:
# inverted index and vocabulary dictionaries
inv_index = {}
voc = {}
# counter for the words in the vocabulary
j = 0

Building the vocabulary. We take all the elements from the tsv files, except the link to build the vocabulary, since all those informations will be needed for the next search engines. The vocabulary will have as keys the words and as values the respective ids.

In [ ]:
for i in range(10000):
    # open the tsv file
    try:
        movie_input = list(pd.pandas.read_csv('tsv_files2/movie_{}.tsv'.format(i), encoding='utf-8', quotechar='"', delimiter='\t'))
    except FileNotFoundError:
        continue

    movie = ' '.join(movie_input[0:-1])
    # do the set since we are not interested in how many times a word appears
    filt_movie = list(set(processing(movie)))
    
    # loop on the words of the filt_movie array
    for word in filt_movie:
        # if the word  is not in voc alredy we add it
        if word not in voc.keys():
            voc[word] = j
            # then we increase the j, which is the word id
            j += 1

    if i % 1000 ==0:
        print('Status: ',i)

#### First index

Building the first index, we take only the first 3 elements of the tsv files, that are : title, intro, plot.

In [ ]:
# import the tsv file
for i in range(10000):
    # open the tsv file
    try:
        movie_input = list(pd.pandas.read_csv('tsv_files2/movie_{}.tsv'.format(i), encoding='utf-8', quotechar='"', delimiter='\t'))
    except FileNotFoundError:
        continue
        
    # for the first part we only need title intro and plot
    # we can join the title, the intro and the plot in the same string
    movie = ' '.join(movie_input[0:3])
    filt_movie = list(set(processing(movie)))
        
    # loop on the words of the filt_movie array
    for word in filt_movie:
        
        # check if the word id is in the keys, if it is update with the new document id       
        if voc[word] in inv_index.keys():
            inv_index[voc[word]].append(i)
        # if the word is not in the inv_index yet then add it 
        else:
            inv_index[voc[word]] = [i]
          
    if i % 1000 == 0:
            print('Status: ',i)

#### Second index


Building the second index. Here we use the tfidf score: $$ tfidf_d = tf_dlog(N/td) $$ where $td_d$ is the term frequency for a given document, $N$ is the number of total documents, and $td$ is the number of documents in which that word appears.For every word from the movies files we count how many times it appears in the document, and divide that by the total number of words in the document, getting the tf term.

In [ ]:
# import the tsv file
for i in range(0,10000):
    # open the tsv file
    try:
        movie_input = list(pd.pandas.read_csv('data\\TSV\\movie_{}.tsv'.format(i+1+20000), encoding='utf-8', quotechar='"', delimiter='\t'))
    except FileNotFoundError:
        continue
        
    # we again only need title intro and plot
    # we can join the title, the intro and the plot in the same string
    movie = ' '.join(movie_input[0:3])
    filt_movie = list(processing(movie))
    #number of words present for the movie
    l = len(filt_movie)    
    # loop on the words of the filt_movie array
    for word in list(set(filt_movie)):
        # index of the word in the array voc
        
        #number of times the word present in the movie file
        n = list(filt_movie).count(word) 
        #calculating term frequency 
        tf = round(n/l,3)
        if voc[word] in inv_index.keys():
            #appending document id and term frequency of the word into the inv_index dictionary
            inv_index[voc[word]].append((i,tf))
        else:
            inv_index[voc[word]] = [(i,tf)]
            
    if i % 100 == 0:
            print('Status: ',i)

Now we calculate the idf term, and make the complete normalized tfidf term.

In [298]:
with open('index1.json', 'r') as myfile1:
    data1=myfile1.read()
inv_index = json.loads(data1)

In [299]:
# total documents ids
docs = []
for x in inv_index.values():
    for y in x:
        docs.append(y[0])
docs = list(set(docs))
# number of total documents
N = len(docs)

In [305]:
# the idf term is the same for all documents of the respective word
for word_id in inv_index:
    # first we count the number of documents the word appears in 
    cont = 0
    for doc in inv_index[word_id]:
        cont += 1
    # then evaluate the idf term  
    idf = np.log(N/cont)
    #print(word_id, idf)
    # then update the inv_index with the tfidf terms.
    for doc in inv_index[word_id]:
        doc[1] *= idf

In [211]:
# given the document id we evaluate its norm
def norm (doc_id):
    norm = 0
    for word_id in inv_index:
        for doc in inv_index[word_id]:
            if doc[0] == doc_id:
                norm += doc[1]*doc[1]
    return math.sqrt(norm)

#### Third index

We decided to use an alternative quantity to the term frequency, the maximum tf normalization. This last one is given by : $$ ntf_d =  a+(1-a)\frac{tf_d}{\text{max}_{tf}(tf_d)} $$ 
where, $a$ is an arbitrary value usually set to 0.4, $tf_d$ the term frequency for that document, and $\text{max}_{tf}(tf_d)$ is the maximum $tf$ component of the document. This term is then multplied for the respective $idf$. The code is similar to the previous index, except that instead of evaluating the index only on the intro and plot we do it on all the intormations, including directors, producers, starring...

In [ ]:
# import the tsv file
for i in range(0,10000):
    # open the tsv file
    try:
        movie_input = list(pd.pandas.read_csv('data\\TSV\\movie_{}.tsv'.format(i+1+20000), encoding='utf-8', quotechar='"', delimiter='\t'))
    except FileNotFoundError:
        continue
        
    # we again only need title intro and plot
    # we can join the title, the intro and the plot in the same string
    movie = ' '.join(movie_input[0:-1])
    filt_movie = list(processing(movie))
    #number of words present for the movie
    l = len(filt_movie)    
    # loop on the words of the filt_movie array
    for word in list(set(filt_movie)):
        # index of the word in the array voc
        
        #number of times the word present in the movie file
        n = list(filt_movie).count(word) 
        #calculating term frequency 
        tf = round(n/l,3)
        if voc[word] in inv_index.keys():
            #appending document id and term frequency of the word into the inv_index dictionary
            inv_index[voc[word]].append((i,tf))
        else:
            inv_index[voc[word]] = [(i,tf)]
            
    if i % 100 == 0:
            print('Status: ',i)

Now we get the complete ntfidf term and normalize it.

In [281]:
with open('vocabulary.json', 'r') as myfile:
    data=myfile.read()
#vocabulary
voc = json.loads(data)

### First engine

We use the first index here. Given an input query we just search for all documents that have all those words in it.

In [406]:
# here is the input query, after that we immediatly preprocess it
query = input()
query = processing(query)

america love story


In [411]:
# we append in result list all documents that have he query words
result = []
# loop on the query words
for word in query:
    # earch row of result corresponds to a word of the query
    w = []
    #  check the id of the word
    word_id = str(voc[word])
    for doc in inv_index[word_id]:
    # then check the documents corresponding to that id
        w.append(doc[0])
    result.append(w)

In [413]:
# since we are interested in documents in which all the words appear we do the intersection
# between all rows of result
a = set(result[0])
for i in range(1,len(result)):
    #a = set(result[i])
    a = a.intersection(set(result[i]))
result = list(a)

In [419]:
# save the documents in output dictionary, given the document id we import the tsv file to get the intro
# and the link to the web page
output = {'Title':[], 'Intro':[], 'Link':[]}
for i in range(len(result)):
    if (result[i] >=10000) & (result[i]< 20000):
        movie = list(pd.pandas.read_csv('tsv_files2/movie_{}.tsv'.format(result[i]), encoding='utf-8', quotechar='"', delimiter='\t'))
        output['Title'].append(movie[0])
        output['Intro'].append(movie[1])
        output['Link'].append(movie[-1])
pd.DataFrame(output).head()

,Title,Intro,Link
0,Toy Story 3,Toy Story 3 is a 2010 American computer-anim...,https://en.wikipedia.org/wiki/Toy_Story_3
1,The Blue Butterfly,The Blue Butterfly ( French: Le papillon bleu...,https://en.wikipedia.org/wiki/The_Blue_Butterfly
2,The American President,The American President is a 1995 American rom...,https://en.wikipedia.org/wiki/The_American_Pre...
3,When Harry Met Sally...,When Harry Met Sally... is a 1989 American ro...,https://en.wikipedia.org/wiki/When_Harry_Met_S...
4,American Beauty,American Beauty is a 1999 American drama film...,https://en.wikipedia.org/wiki/American_Beauty_...


### Second engine

Here we use the second index with the tfidf scores. Given an input query this can be seen as n-dimensional vector (with n the number of words in input) in the space of words. The components of the query vector are all zeros except those corresponding to the input words, which components are all equal to 1/sqrt(n) (the vector is normalized). We use here the cosine similarity to check the "distance" between the query and all our documents, and at the end we print the documents which have the greater similarity with the input query. Every document in the space of words has components that are its tfidf terms, so given this we just need to compute the dot product between 2 vectors and their norm.

In [287]:
# this is the function that returns the similarity

# the document vector is alredy normalized, so we just need to compute the dot product between the query 
# vector and the document
def calc_cos (query, doc_id):
    dot_p = 0
    for words in query:
        word_id = str(voc[words])
        for docs in inv_index[word_id]:
            if docs[0] == doc_id:
                dot_p += docs[1]
    return(dot_p/math.sqrt(len(query)))

In [393]:
# here is the input query, after that we immediatly preprocess it
query = input()
query = processing(query)

music love gun war


In [394]:
# the improve the search time we apply initially the first search engine on the new index

# we append in result list all documents that have the query words
result = []
for word in query:
    w = []
    # first check the id
    word_id = str(voc[word])
    # then check the documents corresponding to that id
    for docs in inv_index[word_id]:
        w.append(docs[0])
    result.append(w)
#result = list(set(result))

In [402]:
a = set(result[0])
for i in range(1,len(result)):
    #a = set(result[i])
    a = a.intersection(set(result[i]))
result = list(a)

In [400]:
result = list(a)
i=0
for doc in list(a):
    norm(doc)
    i+=1
    if i % 10==0:
        print(i)

10
20
30


In [403]:
# then save in a dictionary the documents id and the similarity
sim = {'doc_id': [], 'similarity': []}
# search only in documents that have the query words
for doc in result:
    x = calc_cos(query, doc)
    if x != 0: 
        sim['doc_id'].append(doc)
        sim['similarity'].append(calc_cos(query,doc)/norm(doc))

In [404]:
pd.DataFrame(sim).sort_values(by= ['similarity'], ascending = False).head()

,doc_id,similarity
23,6505,0.071870
11,18500,0.060172
14,717,0.058660
3,2578,0.058145
21,3939,0.054766


At this point we have all the documents id's and the similarities. Just need to print the desired data.

In [296]:
output = {'Title':[], 'Intro':[], 'Link':[], 'Similarity':[]}
for i in range(len(sim['doc_id'])):
    try:
        movie = list(pd.pandas.read_csv('tsv_files2/movie_{}.tsv'.format(sim['doc_id'][i]), encoding='utf-8', quotechar='"', delimiter='\t'))
    except FileNotFoundError:
        continue
    output['Title'].append(movie[0])
    output['Intro'].append(movie[1])
    output['Link'].append(movie[-1])
    output['Similarity'].append(sim['similarity'][i])
pd.DataFrame(output).head()

,Title,Intro,Link,Similarity
0,A Christmas Story,A Christmas Story is a 1983 American Christma...,https://en.wikipedia.org/wiki/A_Christmas_Story,0.012214
1,Deal of the Century,Deal of the Century is a 1983 American comedy...,https://en.wikipedia.org/wiki/Deal_of_the_Century,0.009160
2,National Lampoon's Vacation,"National Lampoon's Vacation, sometimes referr...",https://en.wikipedia.org/wiki/National_Lampoon...,0.006107
3,"Reuben, Reuben","Reuben, Reuben is a 1983 comedy-drama film di...","https://en.wikipedia.org/wiki/Reuben,_Reuben",0.012214
4,The Right Stuff,The Right Stuff is a 1983 American epic histo...,https://en.wikipedia.org/wiki/The_Right_Stuff_...,0.006107
